<a href="https://colab.research.google.com/github/icoqueiro/ProcessoSeletivo_AlbertEinstein/blob/main/Q7_TesteVagaDev_AE_IvanCoqueiro_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Questão 7. Web Frameworks (10 pontos)

Identifique e corrija as vulnerabilidades no seguinte código Flask:

```
# from flask import Flask, request
app = Flask(__name__)

@app.route('/login', methods=['POST'])
def login():
	username = request.form['username']
	password = request.form['password']
	if username == 'admin' and password == '1234':
        return "Acesso concedido"
	return "Acesso negado"

if __name__ == '__main__':
    app.run(debug=True)

```



##Solução

Para iniciar a solução, instalaremos o Ngrok no notebook

In [ ]:
!pip install flask pyngrok

O primeira vulnerabilidade que é possível ser notada é a de que o aplicativo não faz uso de chaves, mas escreve a lógica como hard code. Isso é muito mal visto da perspectiva de código seguro. As alterações seguem na snippet do código principal

In [ ]:
#importa as bibliotecas
from pyngrok import ngrok, conf
from google.colab import userdata
from werkzeug.security import generate_password_hash, check_password_hash #encripta a senha

#aumenta o tempo de timeout
conf.get_default().request_timeout = 150

#registra a chave de acesso do Ngrok
auth_token = userdata.get('auth_token') #usa a sessão de dados sensíveis da google (secrets)
ngrok.set_auth_token(auth_token)


O aplicativo Flask se encontra abaixo. Para evitar maiores quebras de runtime do código, seguirei explicando as alterações via comentários.

In [ ]:
from flask import Flask, request, jsonify
import time
!fuser -k 5000/tcp #matar qualquer processo ouvindo na porta 5000

#cria um dicionario para simular uma base de dados com login
usuarios = {
    "admin": generate_password_hash(userdata.get('password'))  #cria um hash para a senha
}

#inicia o app
app = Flask(__name__)

#tunelamento para evitar brechas de segurança envolvendo tunelamentos clandestinos
ngrok_tunnel = ngrok.connect(5000)
print("* aplicativo funcionando em: *", ngrok_tunnel)

@app.route("/")
def home():
  #introducao
   return "<h1>Olá! </h1>"


#autenticação do login do cliente da plataforma
@app.route("/login", methods=['POST'])
def login():

  #obtém os valores necessários, presente de forma diferente no código original (é  encriptado)
  usuario  = request.form.get('user')
  senha  = request.form.get('password')

  #checa a existência do login e se está correta a senha (hash está condizente com o valor passado). Se correto, conceda o login.
  if usuario in usuarios and check_password_hash(usuarios[usuario], senha):
   return jsonify({"RESULTADO": "Acesso concedido"}), 200
  else:
    return jsonify({"RESULTADO": "Acesso negado"}), 401

#tempo como garantia para inicializar o app
time.sleep(2)
app.run(port=5000,threaded = True, debug=False) #debug fica como falso para não expor nenhuma thread insegura


* aplicativo funcionando em: * NgrokTunnel: "https://e2e6-34-86-186-209.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:02:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:02:57] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:03:04] "POST /login HTTP/1.1" 401 -
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:03:21] "POST /login HTTP/1.1" 401 -
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:03:28] "POST /login HTTP/1.1" 401 -
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:03:41] "POST /login HTTP/1.1" 401 -
INFO:werkzeug:127.0.0.1 - - [16/Feb/2025 19:03:51] "POST /login HTTP/1.1" 200 -
